In [ ]:
# -----------------------------------------------------------------------------
# Portions of this code are adapted from:
#   - https://github.com/TamaraGrossmann/FEM-vs-PINNs.git
#   - Grossmann, T. G., Komorowska, U. J., Latz, J., & Schönlieb, C.-B. (2023).
#     Can Physics-Informed Neural Networks beat the Finite Element Method?
#     arXiv:2302.04107.
# -----------------------------------------------------------------------------

In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# !pip install fenics # for local machine

In [ ]:
import json
import os
import time
from dolfin import *
# from fenics import * # for local machine
import numpy as np
from __future__ import print_function

# Evaluation Points & Number of Meshes

In [ ]:
with open('1D_Poisson_eval_points.json', 'r') as f:
    domain_points = json.load(f)
    domain_points = np.array(domain_points)

# Number of Meshes
mesh_nums = [64, 128, 256, 512, 1024, 2048, 4096]

# Exact Solution

In [ ]:
def u_e(x):
    return x * np.exp(-x * x)

# Boundary Conditions

In [ ]:
def left(point):
    return np.isclose(point[0], 0., atol=1e-5)

def right(point):
    return np.isclose(point[0], 1., atol=1e-5)


def boundary_R(x, on_boundary):
    return on_boundary and np.isclose(x[0], 1)

def boundary_L(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)

# Boundary values
u_L = Expression('0', degree = 1) # symbolic expression (C++ style)
u_R = Expression('exp(-1)', degree = 1) # degree : degree of the polynomial approximation

# Approximate Solution

In [ ]:
# Containers for the results
y_results, times_solve, times_eval, l2_rel\
    = dict({}), dict({}), dict({}), dict({})

n = 0
for mesh_num in mesh_nums:
    # Mesh
    mesh = IntervalMesh(int(mesh_num), 0, 1) # creates "mesh_num" elements in the interval [0, 1]

    ele_type = 'QUAD4'
    cell_type = get_meshio_cell_type(ele_type)
    Lx, Ly = 1., 1.
    meshio_mesh = rectangle_mesh(Nx=32, Ny=32, domain_x=Lx, domain_y=Ly)
    mesh = Mesh(meshio_mesh.points, meshio_mesh.cells_dict[cell_type])

    # Function space
    V = FunctionSpace(mesh, 'CG', 1) # Initialize function space
    # Continuous Galerkin(CG) finite element space of degree 1

    # Dirichlet BC
    bc_L = DirichletBC(V, u_L, boundary_L) # u = "u_L" at x = "boundary_L"
    bc_R = DirichletBC(V, u_R, boundary_R) # Dirichlet BC on the function space V
    bcs = [bc_L, bc_R]

    # Trial and test functions
    u = TrialFunction(V) # it is just a symbolic variable to build the linear system
    v = TestFunction(V)

    # Force term
    f = Expression('6 * x[0] * exp(-x[0]*x[0]) - 4 * (x[0]*x[0]*x[0]) * exp(-x[0]*x[0])', degree = 1)

    # Weak form
    R = dot(grad(u), grad(v)) * dx - f * v * dx
    a = lhs(R) # helper function to get the matrix(operator multiplied to "u") part of weak form
    F = rhs(R) # helper function to get the vector(load) part of weak form

    u = Function(V) # it is a variable to store actual numerical solution

    time_solving = []
    time_evaluation = []
    for i in range(0, 10):
        # Solve
        start_time = time.time()
        solve(a == F, u, bcs, solver_parameters={'linear_solver': 'cg', 'preconditioner': 'ilu'}) # solves for the unknown "u"
        solve_time = time.time()
        time_solving.append(solve_time - start_time)

        # Evaluate
        start_time = time.time()
        u_approx = [u(point) for point in domain_points]
        eval_time = time.time()
        time_evaluation.append(eval_time - solve_time)
        u_approx = np.array(u_approx)

    # Comparison to Ground Truth
    u_true = np.array([u_e(point) for point in domain_points]).squeeze()
    l2 = np.linalg.norm(u_approx - u_true)
    l2_rel_single = l2 / np.linalg.norm(u_true)

    print('Average Solution Time : ', np.mean(time_solving))
    print('Average Evaluation Time : ', np.mean(time_evaluation))
    print('Average Accuracy on a Random Testset of 512 Points : ', l2_rel_single)

    y_results[mesh_num] = u_approx.tolist() # dict with key "mesh_num"
    times_solve[mesh_num] = np.mean(time_solving)
    times_eval[mesh_num] = np.mean(time_evaluation)
    l2_rel[mesh_num] = l2_rel_single

    results = dict({'y_results': y_results,
                    'times_solve': times_solve,
                    'times_eval': times_eval,
                    'l2_rel': l2_rel})

# Save Results & Evaluation
save_dir = './1D_Poisson'
os.makedirs(save_dir, exist_ok = True)

with open(os.path.join(save_dir,'FEM_results.json'), "w") as f:
    json.dump(results, f)